In [103]:
from IPython.display import HTML

HTML('''
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
''')

# Adjustable measurement noise parameters

You can adjust the chance for a spot to be visible here:

In [104]:
p_success = 0.1

You can adjust the number of pixels occupied by a single cell here:

In [105]:
num_pixels = 16 # number of pixels occupied by a typical cells

We will always assumed that parameter search is done in log10-transformed domain. This means that the FIM needs to be transformed accordingly.

In [106]:
par_log_transform = True

See the description below for the meaning of these parameters.

In [107]:
from pypacmensl.sensitivity.multi_sinks import SensFspSolverMultiSinks
import mpi4py.MPI as mpi
import numpy as np

import numpy.linalg as LA
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.stats import binom, poisson, norm
from scipy.special import comb     
from math import exp, log, sqrt

from matplotlib.patches import Ellipse
from matplotlib.ticker import FormatStrFormatter
import ipywidgets as widgets

# For nice plots
import seaborn as sns
sns.set(style='darkgrid')

# Constitutive gene expression model


We will load the time-dependent probability distribution of RNA count from the precomputed FSP solutions. If you want to make changes to the model parameters, please adjust and run the script 'const_gene_expr_fsp.py' again.

In [108]:
fsp_sol_file = np.load('fsp_solutions.npz', allow_pickle=True)
rna_distributions = fsp_sol_file['rna_distributions']
rna_sensitivities = fsp_sol_file['rna_sensitivities']
t_meas = fsp_sol_file['t_meas']
fsp_sol_file.close()

with np.load('bursting_parameters.npz') as par:
    k_on = par['kon']
    k_off = par['koff']
    k_r = par['kr']
    gamma = par['gamma']

theta = np.array([k_on, k_off, k_r, gamma])

## Exact smFISH measurement FIM

In [109]:
# FIM for exact smFISH measurements
fim_exact = np.zeros((len(t_meas), 4, 4))
for itime in range(0, len(t_meas)):
    M = np.zeros((4,4))
    for ip in range(0, 4):
        for jp in range(0, ip+1):
            M[ip, jp] = np.sum(rna_sensitivities[itime][ip]*rna_sensitivities[itime][jp]/
                               np.maximum(rna_distributions[itime], 1.0e-16))
    for ip in range(0,4):
        for jp in range(ip+1,4):
            M[ip, jp] = M[jp, ip]
    fim_exact[itime, : , :] = M
 
np.savez('fim_exact.npz', fim_exact=fim_exact)

## Simulate flow cytometry experimental outcomes

We assume the measured intensity $I$ (in AU) is linked to the actual number of mRNA molecules via the formula
$$I = \kappa \cdot n_{RNA} + \eta_n + \varepsilon_{BG}$$
where $\kappa$ is a scaling constant, $\eta_n \sim \mathcal{N}(0, \sigma^2_{probe}\cdot n)$ is the noise due to variation in probe intensity, and
$\varepsilon_{BG} \sim \mathcal{N}(\mu_{BG}, \sigma^2_{BG})$ is the background noise. 

Recent publications ignore $\sigma_{probe}$, so I will consider the simplified model
$$I = \kappa \cdot n_{RNA} \varepsilon_{BG}.$$

In [110]:
with np.load('flowcyt_noise_parameters.npz') as par:
    kappa = par['kappa'] 
    mu_bg = par['mu_bg'] 
    sigma_bg = par['sigma_bg']

In [111]:
num_cells_flowcyt = 20000
intensity_samples = np.zeros((len(t_meas), num_cells_flowcyt))

for itime in range(0, len(t_meas)):
    xmax = len(rna_distributions[itime])-1
    nrna_samples = np.random.choice(xmax+1, size=(num_cells_flowcyt,), p= rna_distributions[itime]/np.sum(rna_distributions[itime]))
    bg_noise = np.random.normal(loc=mu_bg, scale=sigma_bg, size=(num_cells_flowcyt,))
    intensity_samples[itime, :] = kappa*nrna_samples + bg_noise

## Modeling measurement noise with binomial distribution

The first noise we consider is that coming from inefficient probe binding, leading to the number of visible spots being lower than the actual mRNA copy number in the cell. 

Let $p_{bind}$ be the probability that a given mRNA molecule become visible, the number of spots recorded conditioned on the number of actual mRNA then follows a binomial distribution.

We will call this the "lossy" smFISH measurement.

In [112]:
def BinomialNoiseMatrix(n_max, p_success):
    M = np.zeros((n_max + 1, n_max + 1))  
    for j in range(0, n_max + 1):
            M[:, j] = binom.pmf(np.linspace(0, n_max, n_max+1), j, p_success) 
    return M

n_max = 200
C_binom = BinomialNoiseMatrix(n_max, p_success)

fim_binom = np.zeros((len(t_meas), 4, 4))
for itime in range(0, len(t_meas)):
    M = np.zeros((4,4))
    xmax = len(rna_distributions[itime])-1
    p = C_binom[0:xmax+1, 0:xmax+1]@rna_distributions[itime]
    for ip in range(0, 4):
        sip = C_binom[0:xmax+1, 0:xmax+1]@rna_sensitivities[itime][ip]
        for jp in range(0, ip+1):
            sjp = C_binom[0:xmax+1, 0:xmax+1]@rna_sensitivities[itime][jp]
            M[ip, jp] = np.sum(sip*sjp/np.maximum(1.e-16, p))
    for ip in range(0,4):
        for jp in range(ip+1,4):
            M[ip, jp] = M[jp, ip]
    fim_binom[itime, :, :] = M
np.savez('fim_binom.npz', fim_binom=fim_binom)


## Modeling combinatorial noise from low image resolution

We consider spot miscounting due to low image resolution. Let $S$ be the size of the cell in terms of the pixels it occupies. Ignore for now the variability in cell sizes. Bright spots may clutter within a pixel, making the image looks like there is only one spot when there should actually be several of them.

Given cell size S and n mRNA molecules, the recorded number of spots is given by the formula
$$
P(Y = j | S, n)
=
\frac
{\binom{S}{j}\binom{n-1}{j-1}}
{\binom{S + n - 1}{n}}
$$

This comes from a combinatoric problem: Given $S$ boxes and $n$ apples, how many ways are there to distribute these apples into these boxes such that there are exactly $j$ non-empty boxes.

In [113]:
def LowResNoiseMatrix(n_max):
    M = np.zeros((n_max + 1, n_max + 1))  
    M[0,0] = 1.0
    for j in range(1, n_max + 1):
            M[:, j] = comb(num_pixels, np.arange(0, n_max+1))*comb(j-1, np.arange(0,n_max+1)-1)/comb(num_pixels + j - 1, j)       
    return M

n_max = 200
C_lowres = LowResNoiseMatrix(n_max)

fim_lowres = np.zeros((len(t_meas), 4, 4))
for itime in range(0, len(t_meas)):
    M = np.zeros((4,4))
    xmax = len(rna_distributions[itime])-1
    p = C_lowres[0:xmax+1, 0:xmax+1]@rna_distributions[itime]
    for ip in range(0, 4):
        sip = C_lowres[0:xmax+1, 0:xmax+1]@rna_sensitivities[itime][ip]
        for jp in range(0, ip+1):
            sjp = C_lowres[0:xmax+1, 0:xmax+1]@rna_sensitivities[itime][jp]
            M[ip, jp] = np.sum(sip*sjp/np.maximum(1.e-16, p))
    for ip in range(0,4):
        for jp in range(ip+1,4):
            M[ip, jp] = M[jp, ip]
    fim_lowres[itime, :, :] = M
np.savez('fim_lowres.npz', fim_lowres=fim_lowres)

In [114]:
time_slider = widgets.IntSlider(value=0, min=0, max=len(t_meas)-1, step=1, description='Time:')

def plot_rna_dist_sens(itime):
    fig_smfish = plt.figure()
    fig_smfish.set_tight_layout(True)
    gs = fig_smfish.add_gridspec(3, 2)
    xmax = len(rna_distributions[itime])-1
    p_binom = C_binom[0:xmax+1, 0:xmax+1]@rna_distributions[itime]
    p_lowres = C_lowres[0:xmax+1, 0:xmax+1]@rna_distributions[itime]
    
    fig_smfish.suptitle(f't = {t_meas[itime]:.2f}')
    ax0 = fig_smfish.add_subplot(1,1,1)
    ax0.plot(np.arange(0, xmax+1), rna_distributions[itime], label='True')
    ax0.plot(np.arange(0, xmax+1), p_binom, color='orange', linestyle=':', label='Lossy')
    ax0.plot(np.arange(0, xmax+1), p_lowres, color='magenta', label='Low-resolution')
    ax0.set_xlabel('RNA molecule count')
    ax0.set_ylabel('Probability')
    ax0.legend()
    
#     for ip in range(0,4):
#         sip_binom = C_binom[0:xmax+1, 0:xmax+1]@rna_sensitivities[itime][ip]        
#         sip_lowres = C_lowres[0:xmax+1, 0:xmax+1]@rna_sensitivities[itime][ip]
#         ax1 = fig_smfish.add_subplot(gs[1 + ip // 2, ip % 2])
#         ax1.plot(np.arange(0, xmax+1), rna_sensitivities[itime][ip])
#         ax1.plot(np.arange(0, xmax+1), sip_binom, color='orange', linestyle=':')
#         ax1.plot(np.arange(0, xmax+1), sip_lowres, color='magenta')
#         ax1.set_xlabel('RNA molecule count')
    
    fig_flowcyt, ax = plt.subplots(1,1)
    sns.distplot(intensity_samples[itime, :], ax=ax)
    ax.set_xlabel('Intensity (AU)')
    ax.set_ylabel('Probability')


widgets.interactive(plot_rna_dist_sens, itime=time_slider)

interactive(children=(IntSlider(value=0, description='Time:', max=120), Output()), _dom_classes=('widget-inter…

# 'One-shot' design: where all measurements are done at a single time

## At each measurement time, different types of noisy measurement affect parameter uncertainties differently.

In [115]:
with np.load('fim_exact.npz') as data:
    fim_exact = data['fim_exact']

with np.load('fim_binom.npz') as data:
    fim_binom = data['fim_binom']
    
with np.load('fim_lowres.npz') as data:
    fim_lowres = data['fim_lowres']    
    
with np.load('fim_flowcyt_rep0.npz') as data:
    fim_flowcyt = data['fim_flowcyt']

if par_log_transform:
    for it in range(0, len(t_meas)):
        for i in range(0,4):
            for j in range(0,4):
                fim_exact[it, i, j] *= theta[i]*theta[j]*np.log(10)*np.log(10)
                fim_binom[it, i, j] *= theta[i]*theta[j]*np.log(10)*np.log(10)
                fim_lowres[it, i, j] *= theta[i]*theta[j]*np.log(10)*np.log(10)
                fim_flowcyt[it, i, j] *= theta[i]*theta[j]*np.log(10)*np.log(10)

def plot_conf_ellipse(fim, num_sigma, ax, par_idx, color='red', label='my_ellipse'):
    covmat = np.linalg.inv(fim)
    [eigvals, eigvecs] = np.linalg.eig(covmat[np.ix_([par_idx[0],par_idx[1]], [par_idx[0],par_idx[1]])])
    mu0 = np.log10(theta[par_idx[0]])
    mu1 = np.log10(theta[par_idx[1]])
    sigma0 = np.sqrt(eigvals[0])
    sigma1 = np.sqrt(eigvals[1])

    rot_radian = np.arctan(eigvecs[1,0]/eigvecs[0,0])
    rot_degree = 360*rot_radian/(2*np.pi)
    
    ellipse = patches.Ellipse(xy=(mu0, mu1), width=2*num_sigma*sigma0, height=2*num_sigma*sigma1, angle=rot_degree,
                              fill=False, edgecolor=color, label=label)
    return ax.add_patch(ellipse)

def plot_uncertainties(itime, num_cells_smfish, num_cells_flowcyt):
    fim_mats = [fim_exact[itime, :, :], 
                fim_binom[itime, :, :], 
                fim_lowres[itime, :, :], 
                fim_flowcyt[itime, :, :]]
    
    num_meas = [num_cells_smfish, num_cells_smfish, num_cells_smfish, num_cells_flowcyt]
    
    design_labels = [f'smFISH \n Noise-free \n {num_cells_smfish} cells', 
                        f'smFISH \n Lossy spot \n {num_cells_smfish} cells',
                        f'smFISH \n Low resolution \n {num_cells_smfish} cells',
                        f'flow cytometry \n Additive noise \n {num_cells_flowcyt} cells']
    
    design_colors = ['red', 'darkgreen', 'blue', 'orange']
    
    fig0 = plt.figure()
    fig0.set_tight_layout(True)
    ax0 = fig0.add_subplot(1,1,1)
    
    # The uncertainties in eigendirections of different experiments
    for i in range(0, len(fim_mats)):
        
        [eigval, eigvec] = np.linalg.eig(num_meas[i]*fim_mats[i])
        uncertainties = 1/np.sqrt(eigval)

        ax0.hlines(uncertainties, xmin = (2*i + 1)-0.75, xmax = (2*i + 1) + 0.75, color=design_colors[i], linewidth=2)
        
    ax0.set_yscale('log')
    ax0.set_xticks(2*np.arange(0, len(fim_mats)) + 1)    
    ax0.set_xticklabels(design_labels)
    ax0.set_ylabel('Uncertainty')
    fig0.suptitle(f't = {t_meas[itime]:.2f} \n')
    
    # The 3-sigma ellipses of different experiments
    fig1, axes1 = plt.subplots(1, 2)
    
    for i in range(0, len(fim_mats)):
        plot_conf_ellipse(num_meas[i]*fim_mats[i], 3, axes1[0], [0,1], design_colors[i], design_labels[i])
        plot_conf_ellipse(num_meas[i]*fim_mats[i], 3, axes1[1], [2,3], design_colors[i], design_labels[i]) 

    for i in range(0, 2):
        axes1[i].legend(bbox_to_anchor=(0, 1.02, 1, 0.2), loc='lower left')
        axes1[i].autoscale()
    
time_slider = widgets.IntSlider(value=37, min=1, max=len(t_meas)-1, step=1, description='Time:')
ncells_smfish_input = widgets.IntText(value=2000, description='#smFISH meas.:')
ncells_flowcyt_input = widgets.IntText(value=200000, description='#flow cytometry meas.:')
widgets.interactive(plot_uncertainties, itime=time_slider, num_cells_smfish=ncells_smfish_input, num_cells_flowcyt=ncells_flowcyt_input)

interactive(children=(IntSlider(value=37, description='Time:', max=120, min=1), IntText(value=2000, descriptio…

## One-shot design with D-optimal measurement time

In [119]:
# Information volume across time of different measurement types
nt = fim_flowcyt.shape[0]
    
fimdet_exact = np.zeros((nt,))
fimdet_binom = np.zeros((nt,))
fimdet_lowres = np.zeros((nt,))
fimdet_flowcyt = np.zeros((nt,))

for i in range(0, nt):
    fimdet_exact[i] = np.linalg.det(fim_exact[i,:,:])
    fimdet_binom[i] = np.linalg.det(fim_binom[i,:,:])
    fimdet_lowres[i] = np.linalg.det(fim_lowres[i,:,:])
    fimdet_flowcyt[i] = np.linalg.det(fim_flowcyt[i,:,:])

itopt_exact = np.argmax(fimdet_exact)
itopt_binom = np.argmax(fimdet_binom)
itopt_lowres = np.argmax(fimdet_lowres)
itopt_flowcyt = np.argmax(fimdet_flowcyt)

def plot_dopt(num_cells_smfish, num_cells_flowcyt):    
    fig, ax = plt.subplots(1,1)
    ax.plot(t_meas, num_cells_smfish**4*fimdet_exact, label='Exact smFISH')
    ax.plot(t_meas, num_cells_smfish**4*fimdet_binom, label='Lossy smFISH')
    ax.plot(t_meas, num_cells_smfish**4*fimdet_lowres, label='Low resolution smFISH')
    ax.plot(t_meas, num_cells_flowcyt**4*fimdet_flowcyt, label='Flow cytometry')
    ax.set_yscale('log')
    ax.set_ylabel('D-opt objective')
    ax.set_xlabel('Measurement time')
    ax.legend()
    
    fim_mats = [fim_exact[itopt_exact, :, :], 
                fim_binom[itopt_binom, :, :], 
                fim_lowres[itopt_lowres, :, :], 
                fim_flowcyt[itopt_flowcyt, :, :]]
    
    num_meas = [num_cells_smfish, num_cells_smfish, num_cells_smfish, num_cells_flowcyt]
    
    design_labels = [f'smFISH \n Noise-free \n {num_cells_smfish} cells \n topt = {t_meas[itopt_exact]:.2f}', 
                        f'smFISH \n Lossy spot \n {num_cells_smfish} cells \n topt = {t_meas[itopt_binom]:.2f}',
                        f'smFISH \n Low resolution \n {num_cells_smfish} cells \n topt = {t_meas[itopt_lowres]:.2f}',
                        f'flow cytometry \n Additive noise \n {num_cells_flowcyt} cells \n topt = {t_meas[itopt_flowcyt]:.2f}']
    
    design_colors = ['red', 'darkgreen', 'blue', 'orange']
    
    fig0 = plt.figure()
    fig0.set_tight_layout(True)
    ax0 = fig0.add_subplot(1,1,1)
    
    # The uncertainties in eigendirections of different experiments
    for i in range(0, len(fim_mats)):
        
        [eigval, eigvec] = np.linalg.eig(num_meas[i]*fim_mats[i])
        uncertainties = 1/np.sqrt(eigval)

        ax0.hlines(uncertainties, xmin = (2*i + 1)-0.75, xmax = (2*i + 1) + 0.75, color=design_colors[i], linewidth=2)
        
    ax0.set_yscale('log')
    ax0.set_xticks(2*np.arange(0, len(fim_mats)) + 1)    
    ax0.set_xticklabels(design_labels)
    ax0.set_ylabel('Uncertainty')
    
    # The 3-sigma ellipses of different experiments
    fig1, axes1 = plt.subplots(1, 2)
    
    for i in range(0, len(fim_mats)):
        plot_conf_ellipse(num_meas[i]*fim_mats[i], 3, axes1[0], [0,1], design_colors[i], design_labels[i])
        plot_conf_ellipse(num_meas[i]*fim_mats[i], 3, axes1[1], [2,3], design_colors[i], design_labels[i]) 

    for i in range(0, 2):
        axes1[i].legend(bbox_to_anchor=(0, 1.02, 1, 0.2), loc='lower left')
        axes1[i].autoscale()

ncells_smfish_input = widgets.IntText(value=2000, description='#smFISH meas.:')
ncells_flowcyt_input = widgets.IntText(value=200000, description='#flow cytometry meas.:')
widgets.interactive(plot_dopt, num_cells_smfish=ncells_smfish_input, num_cells_flowcyt=ncells_flowcyt_input)

interactive(children=(IntText(value=2000, description='#smFISH meas.:'), IntText(value=200000, description='#f…